In [76]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
df = pd.read_excel('Percent_Total_Homeless_vf.xlsx')
df.head()

,Unsheltered_perc_tot,TotalBeds_perc_tot,Emergency_perc_tot,Transitional_perc_tot,SafeHaven_perc_tot,PermanentSupportive_perc_tot,PermanentOther_perc_tot,RapidRehousing_perc_tot
0,8.730873,92.979298,71.917192,21.062106,0.000000,54.095410,6.390639,15.391539
1,22.110553,106.155779,72.236181,33.919598,0.000000,48.492462,0.000000,8.417085
2,33.231397,84.301733,52.599388,28.236493,3.465851,177.370031,0.000000,10.805301
3,40.000000,78.019802,51.683168,26.336634,0.000000,47.524752,0.000000,6.138614
4,51.538462,48.461538,27.179487,21.282051,0.000000,10.769231,2.051282,0.000000


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2336 entries, 0 to 2335
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unsheltered_perc_tot          2336 non-null   float64
 1   TotalBeds_perc_tot            2336 non-null   float64
 2   Emergency_perc_tot            2336 non-null   float64
 3   Transitional_perc_tot         2336 non-null   float64
 4   SafeHaven_perc_tot            2336 non-null   float64
 5   PermanentSupportive_perc_tot  2336 non-null   float64
 6   PermanentOther_perc_tot       2336 non-null   float64
 7   RapidRehousing_perc_tot       2336 non-null   float64
dtypes: float64(8)
memory usage: 146.1 KB


In [79]:
pd.cut(df['Unsheltered_perc_tot'], bins=3).value_counts()

(-0.0994, 33.121]    1595
(33.121, 66.242]      524
(66.242, 99.363]      217
Name: Unsheltered_perc_tot, dtype: int64

In [80]:
size_bins=[-1, 33, 66, 100]
group_names = ["0", "1", "2"]

In [81]:
df['Unsheltered_thirds'] = pd.cut(df['Unsheltered_perc_tot'], size_bins, labels=group_names)

In [82]:
df['Unsheltered_thirds'] = df['Unsheltered_thirds'].astype(int)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2336 entries, 0 to 2335
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unsheltered_perc_tot          2336 non-null   float64
 1   TotalBeds_perc_tot            2336 non-null   float64
 2   Emergency_perc_tot            2336 non-null   float64
 3   Transitional_perc_tot         2336 non-null   float64
 4   SafeHaven_perc_tot            2336 non-null   float64
 5   PermanentSupportive_perc_tot  2336 non-null   float64
 6   PermanentOther_perc_tot       2336 non-null   float64
 7   RapidRehousing_perc_tot       2336 non-null   float64
 8   Unsheltered_thirds            2336 non-null   int32  
dtypes: float64(8), int32(1)
memory usage: 155.2 KB


In [84]:
df=df.drop(['Unsheltered_perc_tot'], axis=1)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2336 entries, 0 to 2335
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TotalBeds_perc_tot            2336 non-null   float64
 1   Emergency_perc_tot            2336 non-null   float64
 2   Transitional_perc_tot         2336 non-null   float64
 3   SafeHaven_perc_tot            2336 non-null   float64
 4   PermanentSupportive_perc_tot  2336 non-null   float64
 5   PermanentOther_perc_tot       2336 non-null   float64
 6   RapidRehousing_perc_tot       2336 non-null   float64
 7   Unsheltered_thirds            2336 non-null   int32  
dtypes: float64(7), int32(1)
memory usage: 137.0 KB


In [86]:
# Remove outcome target from features data
y = df.Unsheltered_thirds.values
X = df.drop(columns="Unsheltered_thirds").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [87]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [88]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid", input_dim=7))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
55/55 [==============================] - 0s 1ms/step - loss: 0.7168 - accuracy: 0.4669
Epoch 2/100
55/55 [==============================] - 0s 1ms/step - loss: 0.4464 - accuracy: 0.7540
Epoch 3/100
55/55 [==============================] - 0s 1ms/step - loss: 0.2383 - accuracy: 0.7591
Epoch 4/100
55/55 [==============================] - 0s 1ms/step - loss: 0.0750 - accuracy: 0.7409
Epoch 5/100
55/55 [==============================] - 0s 1ms/step - loss: -0.0513 - accuracy: 0.7409
Epoch 6/100
55/55 [==============================] - 0s 1ms/step - loss: -0.1724 - accuracy: 0.7426
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: -0.3004 - accuracy: 0.7403
Epoch 8/100
55/55 [==============================] - 0s 1ms/step - loss: -0.4639 - accuracy: 0.7460
Epoch 9/100
55/55 [==============================] - 0s 1ms/step - loss: -0.6677 - accuracy: 0.7426
Epoch 10/100
55/55 [==============================] - 0s 1ms/step - loss: -0.9362 - accuracy: 0.7443
Epo

55/55 [==============================] - 0s 1ms/step - loss: -303.8342 - accuracy: 0.7386
Epoch 82/100
55/55 [==============================] - 0s 1ms/step - loss: -311.7792 - accuracy: 0.7426
Epoch 83/100
55/55 [==============================] - 0s 1ms/step - loss: -319.7745 - accuracy: 0.7323
Epoch 84/100
55/55 [==============================] - 0s 1ms/step - loss: -327.5838 - accuracy: 0.7403
Epoch 85/100
55/55 [==============================] - 0s 1ms/step - loss: -335.9468 - accuracy: 0.7386
Epoch 86/100
55/55 [==============================] - 0s 1ms/step - loss: -344.0359 - accuracy: 0.7443
Epoch 87/100
55/55 [==============================] - 0s 1ms/step - loss: -352.3333 - accuracy: 0.7369
Epoch 88/100
55/55 [==============================] - 0s 1ms/step - loss: -360.7350 - accuracy: 0.7426
Epoch 89/100
55/55 [==============================] - 0s 1ms/step - loss: -369.1337 - accuracy: 0.7283
Epoch 90/100
55/55 [==============================] - 0s 1ms/step - loss: -377.8174 - 